<a href="https://colab.research.google.com/github/gomlfx/ApiaryFund/blob/master/XGBR1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install flaml

In [24]:
#designed for Jupyter/kaggle/colab
import pandas as pd
import numpy as np
#import matplotlib for plotting 
import matplotlib.pyplot as plt
%matplotlib inline
#import normalization
from sklearn import preprocessing
import xgboost as xg
from sklearn.ensemble import RandomForestRegressor as rf
#import auto hyperparameter tuning
from flaml import AutoML

In [25]:
 #MT4 csv 
df1=pd.read_csv('EURUSD1440.csv', names=['d','t','eu_o','eu_h','eu_l','eu_c','eu_v'])
df1 = df1.tail(2000)
df2=pd.read_csv('GBPUSD1440.csv', names=['d','t','gu_o','gu_h','gu_l','gu_c','gu_v'])
df2 = df2.tail(2000)

df3=pd.read_csv('USDCAD1440.csv', names=['d','t','uc_o','uc_h','uc_l','uc_c','uc_v'])
df3 = df3.tail(2000)

df4=pd.read_csv('USDCHF1440.csv', names=['d','t','uf_o','uf_h','uf_l','uf_c','uf_v'])
df4 = df4.tail(2000)

df5=pd.read_csv('USDJPY1440.csv', names=['d','t','uj_o','uj_h','uj_l','uj_c','uj_v'])
df5 = df5.tail(2000)

df6=pd.read_csv('USDSEK1440.csv', names=['d','t','us_o','us_h','us_l','us_c','us_v'])
df6 = df6.tail(2000)

#using merge
df_m_1 = df1.merge(df2, on='d')
df_m_2 = df_m_1.merge(df3, on='d')
df_m_3 = df_m_2.merge(df4, on='d')
df_m_4 = df_m_3.merge(df5, on='d')
df_m_5 = df_m_4.merge(df6, on='d')
df_m_5 = df_m_5.drop(columns='t_x')
df_m_5 = df_m_5.drop(columns='t_y')

pd.set_option('display.max_columns', None)
print(df_m_5) 

               d     eu_o     eu_h     eu_l     eu_c    eu_v     gu_o  \
0     2014.03.25  1.38381  1.38470  1.37490  1.38250   77931  1.64960   
1     2014.03.26  1.38249  1.38263  1.37762  1.37830   65921  1.65249   
2     2014.03.27  1.37834  1.37967  1.37283  1.37437   73853  1.65791   
3     2014.03.28  1.37436  1.37729  1.37047  1.37513   62308  1.66098   
4     2014.03.31  1.37531  1.38085  1.37215  1.37735   71330  1.66550   
...          ...      ...      ...      ...      ...     ...      ...   
1993  2021.12.02  1.13191  1.13475  1.12952  1.13013   97454  1.32719   
1994  2021.12.03  1.13005  1.13333  1.12665  1.13088  113218  1.32970   
1995  2021.12.06  1.12997  1.13109  1.12664  1.12848   79067  1.32360   
1996  2021.12.07  1.12821  1.12982  1.12273  1.12681   78783  1.32613   
1997  2021.12.08  1.12650  1.12724  1.12627  1.12704     630  1.32382   

         gu_h     gu_l     gu_c   gu_v     uc_o     uc_h     uc_l     uc_c  \
0     1.65492  1.64807  1.65287  62865  1.118

In [26]:
df_m_5.tail()

,d,eu_o,eu_h,eu_l,eu_c,eu_v,gu_o,gu_h,gu_l,gu_c,gu_v,uc_o,uc_h,uc_l,uc_c,uc_v,uf_o,uf_h,uf_l,uf_c,uf_v,uj_o,uj_h,uj_l,uj_c,uj_v,us_o,us_h,us_l,us_c,us_v
1993,2021.12.02,1.13191,1.13475,1.12952,1.13013,97454,1.32719,1.33331,1.32656,1.32998,37311,1.28133,1.28374,1.27780,1.28048,38280,0.91906,0.92213,0.91780,0.92023,28813,112.725,113.328,112.698,113.185,58505,9.06588,9.09029,9.02448,9.04490,38791
1994,2021.12.03,1.13005,1.13333,1.12665,1.13088,113218,1.32970,1.33094,1.32081,1.32300,38625,1.28018,1.28536,1.27434,1.28488,38257,0.91998,0.92173,0.91655,0.91786,30287,113.003,113.610,112.556,112.828,57050,9.03786,9.17935,9.03547,9.14200,38831
1995,2021.12.06,1.12997,1.13109,1.12664,1.12848,79067,1.32360,1.32858,1.32220,1.32623,34613,1.28291,1.28422,1.27530,1.27560,32905,0.91716,0.92688,0.91656,0.92551,25926,112.863,113.552,112.846,113.484,51711,9.15248,9.16979,8.98850,9.08968,33157
1996,2021.12.07,1.12821,1.12982,1.12273,1.12681,78783,1.32613,1.32890,1.32085,1.32419,33858,1.27557,1.27669,1.26347,1.26383,30074,0.92478,0.92748,0.92306,0.92486,24666,113.451,113.776,113.397,113.568,40041,9.07722,9.13943,9.04948,9.09611,34249
1997,2021.12.08,1.12650,1.12724,1.12627,1.12704,630,1.32382,1.32449,1.32333,1.32438,628,1.26393,1.26464,1.26370,1.26442,687,0.92480,0.92510,0.92368,0.92467,298,113.518,113.590,113.486,113.534,792,9.10286,9.11585,9.09562,9.09670,449


In [27]:
df_m_5.insert(1,'index',df_m_5.index)

In [28]:
#new column: high vs low
df_m_5['eu_h_or_l'] = ((df_m_5.eu_h - df_m_5.eu_o) > (df_m_5.eu_o - df_m_5.eu_l)) 
df_m_5.eu_h_or_l = df_m_5.eu_h_or_l.replace({True:1,False:0})

#new column: close vs yesterday
df_m_5['eu_c_vs_c'] = (df_m_5.eu_c > df_m_5.eu_o) 
df_m_5.eu_c_vs_c = df_m_5.eu_c_vs_c.replace({True:1,False:0})

#new column: shift tomorrow to today
df_m_5['eu_High_next_day'] = df_m_5['eu_h'].shift(-1)
df_m_5['eu_Low_next_day'] = df_m_5['eu_l'].shift(-1)
df_m_5['eu_Close_next_day'] = df_m_5['eu_c'].shift(-1)
df_m_5 = df_m_5.dropna()


In [29]:
df_m_5.tail()

,d,index,eu_o,eu_h,eu_l,eu_c,eu_v,gu_o,gu_h,gu_l,gu_c,gu_v,uc_o,uc_h,uc_l,uc_c,uc_v,uf_o,uf_h,uf_l,uf_c,uf_v,uj_o,uj_h,uj_l,uj_c,uj_v,us_o,us_h,us_l,us_c,us_v,eu_h_or_l,eu_c_vs_c,eu_High_next_day,eu_Low_next_day,eu_Close_next_day
1992,2021.12.01,1992,1.13329,1.13594,1.13023,1.13190,122540,1.32967,1.33515,1.32612,1.32716,42174,1.27766,1.28293,1.27127,1.28225,39839,0.91832,0.92176,0.91690,0.92043,33483,113.087,113.625,112.667,112.767,62416,9.00866,9.08965,8.99263,9.07443,41090,0,0,1.13475,1.12952,1.13013
1993,2021.12.02,1993,1.13191,1.13475,1.12952,1.13013,97454,1.32719,1.33331,1.32656,1.32998,37311,1.28133,1.28374,1.27780,1.28048,38280,0.91906,0.92213,0.91780,0.92023,28813,112.725,113.328,112.698,113.185,58505,9.06588,9.09029,9.02448,9.04490,38791,1,0,1.13333,1.12665,1.13088
1994,2021.12.03,1994,1.13005,1.13333,1.12665,1.13088,113218,1.32970,1.33094,1.32081,1.32300,38625,1.28018,1.28536,1.27434,1.28488,38257,0.91998,0.92173,0.91655,0.91786,30287,113.003,113.610,112.556,112.828,57050,9.03786,9.17935,9.03547,9.14200,38831,0,1,1.13109,1.12664,1.12848
1995,2021.12.06,1995,1.12997,1.13109,1.12664,1.12848,79067,1.32360,1.32858,1.32220,1.32623,34613,1.28291,1.28422,1.27530,1.27560,32905,0.91716,0.92688,0.91656,0.92551,25926,112.863,113.552,112.846,113.484,51711,9.15248,9.16979,8.98850,9.08968,33157,0,0,1.12982,1.12273,1.12681
1996,2021.12.07,1996,1.12821,1.12982,1.12273,1.12681,78783,1.32613,1.32890,1.32085,1.32419,33858,1.27557,1.27669,1.26347,1.26383,30074,0.92478,0.92748,0.92306,0.92486,24666,113.451,113.776,113.397,113.568,40041,9.07722,9.13943,9.04948,9.09611,34249,0,0,1.12724,1.12627,1.12704


In [30]:

df_m_5.isna().sum()

d                    0
index                0
eu_o                 0
eu_h                 0
eu_l                 0
eu_c                 0
eu_v                 0
gu_o                 0
gu_h                 0
gu_l                 0
gu_c                 0
gu_v                 0
uc_o                 0
uc_h                 0
uc_l                 0
uc_c                 0
uc_v                 0
uf_o                 0
uf_h                 0
uf_l                 0
uf_c                 0
uf_v                 0
uj_o                 0
uj_h                 0
uj_l                 0
uj_c                 0
uj_v                 0
us_o                 0
us_h                 0
us_l                 0
us_c                 0
us_v                 0
eu_h_or_l            0
eu_c_vs_c            0
eu_High_next_day     0
eu_Low_next_day      0
eu_Close_next_day    0
dtype: int64

In [31]:
df_m_5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1997 entries, 0 to 1996
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   d                  1997 non-null   object 
 1   index              1997 non-null   int64  
 2   eu_o               1997 non-null   float64
 3   eu_h               1997 non-null   float64
 4   eu_l               1997 non-null   float64
 5   eu_c               1997 non-null   float64
 6   eu_v               1997 non-null   int64  
 7   gu_o               1997 non-null   float64
 8   gu_h               1997 non-null   float64
 9   gu_l               1997 non-null   float64
 10  gu_c               1997 non-null   float64
 11  gu_v               1997 non-null   int64  
 12  uc_o               1997 non-null   float64
 13  uc_h               1997 non-null   float64
 14  uc_l               1997 non-null   float64
 15  uc_c               1997 non-null   float64
 16  uc_v               1997 

In [32]:
#extract row for variable
last_row = df_m_5.iloc[-1:,np.r_[1:34]].values
print(last_row) 

[[1.99600e+03 1.12821e+00 1.12982e+00 1.12273e+00 1.12681e+00 7.87830e+04
  1.32613e+00 1.32890e+00 1.32085e+00 1.32419e+00 3.38580e+04 1.27557e+00
  1.27669e+00 1.26347e+00 1.26383e+00 3.00740e+04 9.24780e-01 9.27480e-01
  9.23060e-01 9.24860e-01 2.46660e+04 1.13451e+02 1.13776e+02 1.13397e+02
  1.13568e+02 4.00410e+04 9.07722e+00 9.13943e+00 9.04948e+00 9.09611e+00
  3.42490e+04 0.00000e+00 0.00000e+00]]


In [33]:
#for high prediction
#select rows to use for x,y
x = df_m_5.iloc[:,np.r_[1:34]].values
print(x)
y = df_m_5.iloc[:,[34]].values
print(y) 

[[0.00000e+00 1.38381e+00 1.38470e+00 ... 5.44310e+04 0.00000e+00
  0.00000e+00]
 [1.00000e+00 1.38249e+00 1.38263e+00 ... 5.24030e+04 0.00000e+00
  0.00000e+00]
 [2.00000e+00 1.37834e+00 1.37967e+00 ... 5.89320e+04 0.00000e+00
  0.00000e+00]
 ...
 [1.99400e+03 1.13005e+00 1.13333e+00 ... 3.88310e+04 0.00000e+00
  1.00000e+00]
 [1.99500e+03 1.12997e+00 1.13109e+00 ... 3.31570e+04 0.00000e+00
  0.00000e+00]
 [1.99600e+03 1.12821e+00 1.12982e+00 ... 3.42490e+04 0.00000e+00
  0.00000e+00]]
[[1.38263]
 [1.37967]
 [1.37729]
 ...
 [1.13109]
 [1.12982]
 [1.12724]]


In [34]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [ ]:
automl_reg = AutoML()
automl_reg.fit(X_train, y_train, estimator_list=["xgboost"], task="regression", time_budget=600, eval_method="auto")

[flaml.automl: 12-07 23:47:05] {1943} INFO - task = regression
[flaml.automl: 12-07 23:47:05] {1945} INFO - Data split method: uniform
[flaml.automl: 12-07 23:47:05] {1949} INFO - Evaluation method: cv
[flaml.automl: 12-07 23:47:05] {2019} INFO - Minimizing error metric: 1-r2
[flaml.automl: 12-07 23:47:05] {2071} INFO - List of ML learners in AutoML Run: ['xgboost']
[flaml.automl: 12-07 23:47:05] {2311} INFO - iteration 0, current learner xgboost
[flaml.automl: 12-07 23:47:05] {2425} INFO - Estimated sufficient time budget=6182s. Estimated necessary time budget=6s.
[flaml.automl: 12-07 23:47:05] {2505} INFO -  at 0.6s,	estimator xgboost's best error=38.6448,	best estimator xgboost's best error=38.6448
[flaml.automl: 12-07 23:47:05] {2311} INFO - iteration 1, current learner xgboost
[flaml.automl: 12-07 23:47:06] {2505} INFO -  at 0.7s,	estimator xgboost's best error=38.6448,	best estimator xgboost's best error=38.6448
[flaml.automl: 12-07 23:47:06] {2311} INFO - iteration 2, current le

In [36]:
from xgboost import XGBRegressor

#paste retrained model
sugg_reg_high_param = XGBRegressor(colsample_bylevel=0.7735847515513625, colsample_bytree=1.0,
             grow_policy='lossguide', learning_rate=0.01578582627082461,
             max_depth=0, max_leaves=14, min_child_weight=1.107304504895782,
             n_estimators=907, n_jobs=-1, reg_alpha=0.010153693558416055,
             reg_lambda=0.034966802423285405, subsample=1.0, tree_method='hist',
             use_label_encoder=False, verbosity=0)
#paste finished
sugg_reg_high_param.fit(X_train,y_train)
y_pred_h=sugg_reg_high_param.predict(last_row)
y_pred_h

array([1.1282575], dtype=float32)

In [38]:
y_pred = sugg_reg_high_param.predict(X_test)

In [39]:
def mean_absolute_percentage_error(y_true,y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test,y_pred)

In [40]:
#for regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)
print('MedAE', metrics.median_absolute_error(y_test,y_pred)) 

MAE: 0.002974342013804116
MSE: 1.7577455865652052e-05
RMSE: 0.004192547658125313
MAPE: 6.24064703490447
MedAE 0.002272793540954554


In [41]:
#for low prediction
#select rows to use for x,y
x = df_m_5.iloc[:,np.r_[1:34]].values
print(x)
y = df_m_5.iloc[:,[35]].values
print(y) 

[[0.00000e+00 1.38381e+00 1.38470e+00 ... 5.44310e+04 0.00000e+00
  0.00000e+00]
 [1.00000e+00 1.38249e+00 1.38263e+00 ... 5.24030e+04 0.00000e+00
  0.00000e+00]
 [2.00000e+00 1.37834e+00 1.37967e+00 ... 5.89320e+04 0.00000e+00
  0.00000e+00]
 ...
 [1.99400e+03 1.13005e+00 1.13333e+00 ... 3.88310e+04 0.00000e+00
  1.00000e+00]
 [1.99500e+03 1.12997e+00 1.13109e+00 ... 3.31570e+04 0.00000e+00
  0.00000e+00]
 [1.99600e+03 1.12821e+00 1.12982e+00 ... 3.42490e+04 0.00000e+00
  0.00000e+00]]
[[1.37762]
 [1.37283]
 [1.37047]
 ...
 [1.12664]
 [1.12273]
 [1.12627]]


In [42]:
#for low prediction
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [43]:
automl_reg = AutoML()
automl_reg.fit(X_train, y_train, estimator_list=["xgboost"], task="regression", time_budget=600, eval_method="auto")

[flaml.automl: 12-08 00:10:00] {1943} INFO - task = regression
[flaml.automl: 12-08 00:10:00] {1945} INFO - Data split method: uniform
[flaml.automl: 12-08 00:10:00] {1949} INFO - Evaluation method: cv
[flaml.automl: 12-08 00:10:00] {2019} INFO - Minimizing error metric: 1-r2
[flaml.automl: 12-08 00:10:00] {2071} INFO - List of ML learners in AutoML Run: ['xgboost']
[flaml.automl: 12-08 00:10:00] {2311} INFO - iteration 0, current learner xgboost
[flaml.automl: 12-08 00:10:00] {2425} INFO - Estimated sufficient time budget=1148s. Estimated necessary time budget=1s.
[flaml.automl: 12-08 00:10:00] {2505} INFO -  at 0.1s,	estimator xgboost's best error=37.8716,	best estimator xgboost's best error=37.8716
[flaml.automl: 12-08 00:10:00] {2311} INFO - iteration 1, current learner xgboost
[flaml.automl: 12-08 00:10:00] {2505} INFO -  at 0.2s,	estimator xgboost's best error=37.8716,	best estimator xgboost's best error=37.8716
[flaml.automl: 12-08 00:10:00] {2311} INFO - iteration 2, current le

In [44]:
from xgboost import XGBRegressor

#paste retrained model
sugg_reg_low_param = XGBRegressor(colsample_bylevel=0.9592879588141041, colsample_bytree=1.0,
             grow_policy='lossguide', learning_rate=0.013488593217061895,
             max_depth=0, max_leaves=5, min_child_weight=0.43462849479807825,
             n_estimators=1038, n_jobs=-1, reg_alpha=0.02182073928249339,
             reg_lambda=0.36764156052221886, subsample=0.9211577277604256,
             tree_method='hist', use_label_encoder=False, verbosity=0)
#paste finished
sugg_reg_low_param.fit(X_train,y_train)
y_pred_l=sugg_reg_low_param.predict(last_row)
y_pred_l

array([1.1232481], dtype=float32)

In [45]:
#for low pred 
y_pred = sugg_reg_low_param.predict(X_test)
mape = mean_absolute_percentage_error(y_test,y_pred)

In [46]:
#for low pred regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)
print('MedAE', metrics.median_absolute_error(y_test,y_pred)) 

MAE: 0.0029246597786585444
MSE: 1.6349558016580218e-05
RMSE: 0.004043458670071974
MAPE: 6.448148457319131
MedAE 0.00225603111267092


In [47]:
#for close prediction
#select rows to use for x,y
x = df_m_5.iloc[:,np.r_[1:34]].values
print(x)
y = df_m_5.iloc[:,[36]].values
print(y) 

[[0.00000e+00 1.38381e+00 1.38470e+00 ... 5.44310e+04 0.00000e+00
  0.00000e+00]
 [1.00000e+00 1.38249e+00 1.38263e+00 ... 5.24030e+04 0.00000e+00
  0.00000e+00]
 [2.00000e+00 1.37834e+00 1.37967e+00 ... 5.89320e+04 0.00000e+00
  0.00000e+00]
 ...
 [1.99400e+03 1.13005e+00 1.13333e+00 ... 3.88310e+04 0.00000e+00
  1.00000e+00]
 [1.99500e+03 1.12997e+00 1.13109e+00 ... 3.31570e+04 0.00000e+00
  0.00000e+00]
 [1.99600e+03 1.12821e+00 1.12982e+00 ... 3.42490e+04 0.00000e+00
  0.00000e+00]]
[[1.3783 ]
 [1.37437]
 [1.37513]
 ...
 [1.12848]
 [1.12681]
 [1.12704]]


In [48]:
#for close prediction
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [ ]:
automl_reg = AutoML()
automl_reg.fit(X_train, y_train, estimator_list=["xgboost"], task="regression", time_budget=600, eval_method="auto")

[flaml.automl: 12-08 00:27:31] {1943} INFO - task = regression
[flaml.automl: 12-08 00:27:31] {1945} INFO - Data split method: uniform
[flaml.automl: 12-08 00:27:31] {1949} INFO - Evaluation method: cv
[flaml.automl: 12-08 00:27:31] {2019} INFO - Minimizing error metric: 1-r2
[flaml.automl: 12-08 00:27:31] {2071} INFO - List of ML learners in AutoML Run: ['xgboost']
[flaml.automl: 12-08 00:27:31] {2311} INFO - iteration 0, current learner xgboost
[flaml.automl: 12-08 00:27:31] {2425} INFO - Estimated sufficient time budget=1007s. Estimated necessary time budget=1s.
[flaml.automl: 12-08 00:27:31] {2505} INFO -  at 0.1s,	estimator xgboost's best error=39.4932,	best estimator xgboost's best error=39.4932
[flaml.automl: 12-08 00:27:31] {2311} INFO - iteration 1, current learner xgboost
[flaml.automl: 12-08 00:27:31] {2505} INFO -  at 0.2s,	estimator xgboost's best error=39.4932,	best estimator xgboost's best error=39.4932
[flaml.automl: 12-08 00:27:31] {2311} INFO - iteration 2, current le

In [ ]:
from xgboost import XGBRegressor

#paste retrained model
sugg_reg_close_param = XGBRegressor(colsample_bylevel=0.9592879588141041, colsample_bytree=1.0,
             grow_policy='lossguide', learning_rate=0.013488593217061895,
             max_depth=0, max_leaves=5, min_child_weight=0.43462849479807825,
             n_estimators=1038, n_jobs=-1, reg_alpha=0.02182073928249339,
             reg_lambda=0.36764156052221886, subsample=0.9211577277604256,
             tree_method='hist', use_label_encoder=False, verbosity=0)
#paste finished
sugg_reg_close_param.fit(X_train,y_train)
y_pred_l=sugg_reg_close_param.predict(last_row)
y_pred_l

array([1.1232481], dtype=float32)

In [ ]:
#for close prediction
y_pred = sugg_reg_close_param.predict(X_test)
mape = mean_absolute_percentage_error(y_test,y_pred)

In [ ]:
#for close regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)
print('MedAE', metrics.median_absolute_error(y_test,y_pred)) 
#print('MdAPE', metrics)

MAE: 0.004857444772783917
MSE: 4.0246388605608623e-05
RMSE: 0.0063440041460901195
MAPE: 6.68317053335399
MedAE 0.003820765228271461


In [ ]:
print('Today the predictions are:')
print('high', y_pred_h)
print('close', y_pred_c)
print('low', y_pred_l)
print('Please use these as polarities.')

Today the predictions are:
high [1.1309729]
close [1.1257324]
low [1.1235718]
Please use these as polarities.
